In [1]:
!pip install catboost

In [2]:
import numpy as np
import pandas as pd

import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, f1_score, accuracy_score, roc_auc_score
%matplotlib inline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Iris


In [4]:
df1_bin = pd.read_csv('iris_setosa_bin.csv')
df1_bin.drop(['sample_id'], axis=1, inplace = True)
df1_bin

,"sepal_length(4.3, 5.8)","sepal_length(5.8, 7.3)","sepal_length(7.3, 8.8)","sepal_width(2.0, 3.5)","sepal_width(3.5, 5.0)","petal_length(1.0, 2.5)","petal_length(2.5, 4.0)","petal_length(4.0, 5.5)","petal_length(5.5, 7.0)","petal_width(0.1, 1.6)","petal_width(1.6, 3.1)",is_setosa
0,True,False,False,False,True,True,False,False,False,True,False,True
1,True,False,False,True,False,True,False,False,False,True,False,True
2,True,False,False,True,False,True,False,False,False,True,False,True
3,True,False,False,True,False,True,False,False,False,True,False,True
4,True,False,False,False,True,True,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
145,False,True,False,True,False,False,False,True,False,False,True,False
146,False,True,False,True,False,False,False,True,False,False,True,False
147,False,True,False,True,False,False,False,True,False,False,True,False
148,False,True,False,True,False,False,False,True,False,False,True,False


In [5]:
y_feat = 'is_setosa'

In [6]:
df_train, df_test = train_test_split(df1_bin, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train.describe()

,"sepal_length(4.3, 5.8)","sepal_length(5.8, 7.3)","sepal_length(7.3, 8.8)","sepal_width(2.0, 3.5)","sepal_width(3.5, 5.0)","petal_length(1.0, 2.5)","petal_length(2.5, 4.0)","petal_length(4.0, 5.5)","petal_length(5.5, 7.0)","petal_width(0.1, 1.6)","petal_width(1.6, 3.1)"
count,105,105,105,105,105,105,105,105,105,105,105
unique,2,2,2,2,2,2,2,2,2,2,2
top,False,False,False,True,False,False,False,False,False,True,False
freq,54,58,98,90,90,71,96,65,83,64,64


In [7]:
X_test.describe()

,"sepal_length(4.3, 5.8)","sepal_length(5.8, 7.3)","sepal_length(7.3, 8.8)","sepal_width(2.0, 3.5)","sepal_width(3.5, 5.0)","petal_length(1.0, 2.5)","petal_length(2.5, 4.0)","petal_length(4.0, 5.5)","petal_length(5.5, 7.0)","petal_width(0.1, 1.6)","petal_width(1.6, 3.1)"
count,45,45,45,45,45,45,45,45,45,45,45
unique,2,2,2,2,2,2,2,2,2,2,2
top,False,False,False,True,False,False,False,False,False,True,False
freq,23,23,44,36,36,29,43,24,39,34,34


In [8]:
results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
}

models = {
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'KNN': {'n_neighbors': [3, 5, 7, 10]},
    'DecisionTree': {'max_depth': [None, 5, 7, 10]},
    'RandomForest': {'n_estimators': [10, 20, 35, 50], 'max_depth': [None, 2, 5, 10]},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

In [9]:
for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X_train, y_train)

    best_model = clf.best_estimator_

    acc_scores = cross_val_score(best_model, X_test, y_test, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_model, X_test, y_test, cv=5, scoring=make_scorer(f1_score, average='macro'))

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score
0                 KNN     0.956     0.955
1        DecisionTree     1.000     1.000
2        RandomForest     1.000     1.000
3  LogisticRegression     1.000     1.000


# 70k+ Job Applicants Data (Human Resource)


In [10]:
df2_bin = pd.read_csv('job_bin.csv')
df2_bin.drop(['Unnamed: 0'], axis=1, inplace = True)
df2_bin

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Employed,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,...,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
0,True,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,True,True,True,True,True,True,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2,True,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,True,False
3,True,True,True,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
4,False,True,True,True,True,False,True,False,False,False,...,False,False,False,False,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,True,True,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
996,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
997,False,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,True,True,True,True
998,True,True,True,True,True,False,True,False,False,False,...,False,False,False,False,False,True,True,True,True,False


In [11]:
y_feat = 'Employed'
df_train, df_test = train_test_split(df2_bin, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train.describe()

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,EdLevel_PhD,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
count,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700
unique,2,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
freq,604,700,677,700,700,490,683,668,591,672,373,691,678,571,627,604,422,450,419,377


In [12]:
X_test.describe()

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,EdLevel_PhD,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
count,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
unique,2,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True
freq,263,300,291,300,300,197,288,289,258,283,151,296,290,244,270,263,172,172,168,151


In [13]:
# bool -> 0, 1
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [14]:
results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
}

models = {
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'KNN': {'n_neighbors': [3, 5, 7, 10]},
    'DecisionTree': {'max_depth': [None, 5, 7, 10]},
    'RandomForest': {'n_estimators': [10, 20, 35, 50], 'max_depth': [None, 2, 5, 10]},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

In [15]:
for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X_train, y_train)

    acc_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring=make_scorer(f1_score, average='macro'))

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score
0                 KNN     0.783     0.782
1        DecisionTree     0.783     0.781
2        RandomForest     0.810     0.805
3  LogisticRegression     0.820     0.818


# Wine

In [16]:
!pip3 install -U ucimlrepo

In [17]:
from ucimlrepo import fetch_ucirepo
from sklearn.utils import shuffle

wine = fetch_ucirepo(id=109)
X = wine.data.features
y = wine.data.targets

In [18]:
#make binary from multiclass

idx = y[y == 3].dropna().index

X = X.drop(idx)
y = y.drop(idx)

In [19]:
y['class'] = y['class'] == 1

In [20]:
sh_data = shuffle(pd.concat([X, y], axis=1))
df = sh_data.iloc[:129]
X = df.drop(columns=['class'], axis=1)
y = df['class']

In [21]:
#binarization by median value

for col in X.columns:
    median_val = X[col].median()
    X.loc[:, col] = X[col] >= median_val

In [22]:
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

# bool -> 0, 1
X = X.astype(int)
y = y.astype(int)

In [23]:
X

,Alcohol,Malicacid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,0D280_0D315_of_diluted_wines,Proline
0,1,0,1,1,1,1,1,0,1,1,1,0,1
1,1,0,0,0,1,1,1,1,1,1,0,1,1
2,0,0,1,1,0,1,1,0,1,0,0,1,0
3,1,1,0,0,1,1,1,1,0,1,0,1,0
4,0,1,1,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,1,1,1,0,1,1,1,0,1,1,0,1,1
125,1,1,1,1,1,1,1,1,1,1,1,0,1
126,0,0,0,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,1,0,0,0,1,0,1,1,1


In [24]:
y

0      1
1      1
2      0
3      1
4      0
      ..
124    1
125    1
126    0
127    0
128    0
Name: class, Length: 129, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.describe()

,Alcohol,Malicacid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,0D280_0D315_of_diluted_wines,Proline
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.511111,0.511111,0.500000,0.522222,0.533333,0.544444,0.500000,0.600000,0.466667,0.488889,0.522222,0.500000,0.466667
std,0.502677,0.502677,0.502801,0.502304,0.501683,0.500811,0.502801,0.492642,0.501683,0.502677,0.502304,0.502801,0.501683
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.500000,1.000000,1.000000,1.000000,0.500000,1.000000,0.000000,0.000000,1.000000,0.500000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
X_test.describe()

,Alcohol,Malicacid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,0D280_0D315_of_diluted_wines,Proline
count,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000
mean,0.487179,0.512821,0.512821,0.461538,0.487179,0.410256,0.512821,0.461538,0.589744,0.538462,0.487179,0.564103,0.589744
std,0.506370,0.506370,0.506370,0.505035,0.506370,0.498310,0.506370,0.505035,0.498310,0.505035,0.506370,0.502356,0.498310
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
}

models = {
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'KNN': {'n_neighbors': [3, 5, 7, 10]},
    'DecisionTree': {'max_depth': [None, 5, 7, 10]},
    'RandomForest': {'n_estimators': [10, 20, 35, 50], 'max_depth': [None, 2, 5, 10]},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

In [28]:
for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X_train, y_train)

    acc_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring=make_scorer(f1_score, average='macro'))

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score
0                 KNN     0.871     0.870
1        DecisionTree     0.875     0.870
2        RandomForest     0.925     0.896
3  LogisticRegression     0.925     0.924
